![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [5]:
# === 第1步：筛选符合正则的文件（支持 max_depth） ===
import os
import re
from pathlib import Path
import json
def find_files(base_dir=".", pattern=None, max_depth=None):
    """
    遍历目录并筛选出符合正则的文件
    :param base_dir: 要遍历的根目录
    :param pattern: 文件名匹配正则表达式
    :param max_depth: 最大遍历深度（None 表示不限制）
    """
    base_path = Path(base_dir).resolve()
    regex = re.compile(pattern) if pattern else None
    matched_files = []

    for root, dirs, files in os.walk(base_path):
        # 计算当前深度
        depth = len(Path(root).relative_to(base_path).parts)
        if max_depth is not None and depth > max_depth:
            dirs[:] = []  # 不再深入
            continue

        for f in files:
            fpath = Path(root) / f
            if regex is None or regex.search(f):
                matched_files.append(fpath)

    print(f"在目录 {base_path} 中找到 {len(matched_files)} 个匹配文件：")
    matched_files = sorted(matched_files, key=lambda x: str(x).lower())

    for f in matched_files:
        print("   ", f)
    print("-" * 60)
    return matched_files


# === 第2步：并行生成 Base64 下载链接 ===
import base64
from concurrent.futures import ThreadPoolExecutor, as_completed
from IPython.display import display, HTML

def generate_base64_link(file_path):
    """
    读取单个文件并生成 Base64 下载链接
    """
    file_name = os.path.basename(file_path)
    with open(file_path, "rb") as f:
        encoded = base64.b64encode(f.read()).decode()
    return f'<a download="{file_path}" href="data:application/zip;base64,{encoded}">下载 {file_name}</a>'

def create_base64_links_parallel(files, max_workers=8):
    """
    并行生成 Base64 下载链接并显示
    """
    if not files:
        return

    print(f" 开始并行生成 {len(files)} 个文件的 Base64 链接...")
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(generate_base64_link, f): f for f in files}

        for future in as_completed(futures):
            file_path = futures[future]
            try:
                link_html = future.result()
                display(HTML(link_html))
            except Exception as e:
                print(f"  生成 {file_path} 链接失败: {e}")

    print(f" 全部完成，共生成 {len(files)} 个链接。")

def split_list(lst, chunk_size):
    """将列表按指定大小分块"""
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]

file_list=[]
#file_list += find_files("/Data/future/comex/factor_files", pattern=r".*20251021.zip", max_depth=1)  # 下载一个即可，解压
#file_list += find_files("/Data/future/cme/factor_files", pattern=r".*20251021.zip", max_depth=1)  # 下载一个即可，解压
#file_list += find_files("/Data/future/comex/map_files", pattern=r".*20251021.zip", max_depth=1) # 下载一个即可，解压
#file_list += find_files("/Data/future/cme/map_files", pattern=r".*20251021.zip", max_depth=1)  # 下载一个即可，解压

# file_list += find_files("/Data/future/comex/daily", pattern=r"(^gc_)|(^si_).*", max_depth=1)
file_list += find_files("/Data/future_old/cme/daily", pattern=r"(^nq_)|(^es_).*", max_depth=1)


# year = 202201
# file_list += find_files("/Data/future/cme/second/nq", pattern=rf"^{year}.*e.zip", max_depth=1)




print(f"共计 {len(file_list)} 个匹配文件")
#file_list += find_files("/Data/future/cme/", pattern=rf".*", max_depth=0)

#batches = list(split_list(file_list, 10))
#create_base64_links_parallel(file_list, max_workers=8

In [ ]:
import os
def list_dir_with_max_depth_corrected(start_dir='.', max_depth=2):
    """ 
    修正后的版本，更准确地匹配 ls --max-depth=N 的语义。
    ls --max-depth=N 会列出所有深度 <= N 的目录和文件。
    """
    start_dir = os.path.abspath(start_dir)
    
    for root, dirs, files in os.walk(start_dir, topdown=True):
        # 相对路径和深度计算
        relative_path = os.path.relpath(root, start_dir)
        if root == start_dir:
            current_depth = 0
        else:
            current_depth = relative_path.count(os.sep) + 1 # 目录的深度
            
        # 1. 如果当前目录深度超过 max_depth，则跳过
        if current_depth > max_depth:
            continue
            
        # 2. 打印当前目录 (如果其深度 <= max_depth)
        indent = "  " * current_depth
        
        # 仅在深度允许时打印
        if current_depth <= max_depth:
             # 打印目录本身
            dir_to_print = os.path.basename(root) if root != start_dir else os.path.basename(start_dir)
            
            # 使用相对路径，更像 ls 的输出风格
            print(f"{indent}[{relative_path}/]")
            
            # 3. 打印当前目录下的文件
            for file_name in files:
                file_depth = current_depth + 1
                if file_depth <= max_depth:
                    file_indent = "  " * file_depth
                    print(f"{file_indent}{file_name}")

        # 4. 剪枝 (Pruning)：如果当前深度已达到最大深度，则清空 dirs 列表，阻止进入子目录
        # 注意：这里的 max_depth 是针对**目录**的深度。
        # 如果 max_depth=2，深度 2 的目录 (如 dir1/dir1_1) 会被列出，
        # 但它下面的子目录 (深度 3) 不会再被访问。
        if current_depth >= max_depth:
            del dirs[:]


list_dir_with_max_depth_corrected(start_dir='/Data/future_old/cme/daily', max_depth=2)

In [6]:
create_base64_links_parallel(file_list, max_workers=2)